In [25]:
import tensorflow as tf
import numpy as np


In [26]:
import io,os,sys,types
from IPython import get_ipython #for import notebook
from nbformat import read #for import notebook
from IPython.core.interactiveshell import InteractiveShell #for import notebook

In [27]:
def find_notebook(fullname, path=None): #for import notebook
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [28]:
class NotebookLoader(object): #for import notebook
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [30]:
sys.meta_path.append(NotebookFinder())
import myprepdata

importing Jupyter notebook from myprepdata.ipynb
[('he', 0.9606667160987854),
 ('She', 0.7691636085510254),
 ('herself', 0.7592681646347046),
 ('He', 0.7463222742080688),
 ('nobody', 0.7147209644317627),
 ('Picard', 0.6965363025665283),
 ('himself', 0.6882157921791077),
 ('him', 0.6789597272872925),
 ('her', 0.6740684509277344),
 ('Kirk', 0.667111337184906)]
0
1
4
10
21
23
314
977
2771
11984
12819
112232
161069
414849
529926
708644
1079277
1916152
1949607
1949617
2191402
4391595


In [2]:
class attention_decoder(object):
    def __init__(self, layers, directions  , maxv, hiddensizes , input_v , output_v, en_state, teacher_forcing):
        
        self.grustate = en_state
        self.output = tf.Variable(tf.zeros([self.hiddensize]),dtype='float32')
        self.W = tf.Variable(tf.zeros([self.hiddensizes*2, maxv]), dtype='float32')
        i = 0
        with tf.variable_scope('input') as scope:
            self.input = tf.placeholder(dtype='float32', shape=[1, self.maxl, self.hiddensize])
            
        tf.slice(self.input,[0,0,0],[0,i,self.hiddensize]
        with tf.variable_scope('gru') as scope:
            self.gru = tf.nn.rnn_cell.GRUCell(num_units = self.hiddensize)
            if teacher_forcing:
                while(i < maxv): #teacher forcing
                    self.grustate, self.output =self.gru.call(self.grustate,tf.slice(output_v,[0,0,0],[0,i,self.hiddensize] ))
                    i = i + 1
                i = 0
            else:
                while(i < maxv): #not teacher forcing
                    self.grustate, self.output =self.gru.call(self.grustate, self.output ))
                    word = myprepdata.findwordsfromvec(self.output)
                    self.output = myprepdata.findvecfromwords(word)
                    attn = tf.concat(0,[self.output, self.grustate])#concat and linear
                    attn = tf.matmul(attn , self.W) # needs W initialize
                    attn = tf.nn.softmax(attn)
                    bmm = tf.matmul(attn, input_v)
                    
                    i = i +1
                i = 0

                
        with tf.variable_scope('decoder') as scope:
            init_hid = tf.Variable(tf.zeros([100], tf.float32),dtype="float32")
            decell = tf.nn.rnn_cell.GRUCell(num_units=100)
            deoutputs, _destates = tf.nn.dynamic_rnn(decell, inputs = inputv, dtype=tf.float32)
   
    def find_mostsim(self,inputv):
        return
        
    def gru_decoder(self):
        return
    
    def hidden_init(self):
        return sess.run(tf.global_variables_initializer())

IndentationError: expected an indented block (<ipython-input-2-c9de766d5ffd>, line 27)

In [ ]:
gru = tf.nn.rnn_cell.GRUCell

In [ ]:
#c = tf.add(c, b)

a = tf.Variable([[1,0,0,0]],dtype="float32")
b = tf.Variable([[0,1,0,0]],dtype="float32")
k = tf.assign(b,tf.subtract(b,a))
c = tf.Variable([[0,0,1,0]],dtype="float32")
print(a[0][2])
i = 0
while(i < 10):
    c = tf.add(b,c)
    c = tf.add(b,c)
    i = i + 1
#b = tf.add(a,b)
ppp = tf.assign(a,tf.subtract(b,a))
qqq = tf.assign(b,tf.subtract(b,c))
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [5]:
print(sess.run([b]))
print(sess.run([c]))
print(sess.run([ppp]))
print(sess.run([b]))
print(sess.run([ppp]))
print(sess.run([a]))
print(sess.run([b]))
print(sess.run([ppp]))
print(sess.run([ppp]))

[array([[ 0.,  1.,  0.,  0.]], dtype=float32)]
[array([[  0.,  20.,   1.,   0.]], dtype=float32)]
[array([[-1.,  1.,  0.,  0.]], dtype=float32)]
[array([[ 0.,  1.,  0.,  0.]], dtype=float32)]
[array([[ 1.,  0.,  0.,  0.]], dtype=float32)]
[array([[ 1.,  0.,  0.,  0.]], dtype=float32)]
[array([[ 0.,  1.,  0.,  0.]], dtype=float32)]
[array([[-1.,  1.,  0.,  0.]], dtype=float32)]
[array([[ 1.,  0.,  0.,  0.]], dtype=float32)]


In [ ]:
print(sess.run([c]))


In [ ]:
print(sess.run([c]))

In [ ]:
p = tf.assign(c, b)

In [ ]:
zero_tsr = tf.Variable([0,0])
tf.assign(zero_tsr,[4,5])


In [ ]:
sess.run([zero_tsr])

In [ ]:
tf.train.GradientDescentOptimizer()

In [22]:
a = tf.constant([[[1, 1, 1], [2, 2, 2]], [[3, 3, 3], [4, 4, 4]], [[5, 5, 5], [6, 6, 6]]], dtype = 'float32')


b = tf.slice(a , [0, 0, 0], [3, 2, 3])
b1 = tf.slice(a , [0, 0, 1], [1, 1, 2])
b2 = tf.slice(a , [0, 0, 0], [1, 1, 3])
b3 = tf.slice(a , [0, 0, 1], [-1, -1, 2])
b4 = tf.slice(a , [0, 0, 0], [1, 0, 2])
b5 = tf.slice(a , [0, 0, 0], [1, 0, 3])
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run([b,b1,b2,b3,b4,b5])


[array([[[ 1.,  1.,  1.],
         [ 2.,  2.,  2.]],
 
        [[ 3.,  3.,  3.],
         [ 4.,  4.,  4.]],
 
        [[ 5.,  5.,  5.],
         [ 6.,  6.,  6.]]], dtype=float32),
 array([[[ 1.,  1.]]], dtype=float32),
 array([[[ 1.,  1.,  1.]]], dtype=float32),
 array([[[ 1.,  1.],
         [ 2.,  2.]],
 
        [[ 3.,  3.],
         [ 4.,  4.]],
 
        [[ 5.,  5.],
         [ 6.,  6.]]], dtype=float32),
 array([], shape=(1, 0, 2), dtype=float32),
 array([], shape=(1, 0, 3), dtype=float32)]